In [48]:
import yfinance as yf
import pandas as pd
import pandas_ta
import matplotlib.pyplot as plt
import numpy as np

In [49]:
data = pd.read_csv('./data/Raw_Stock_Data.csv')
data

,Date,Index,Adj Close,Close,High,Low,Open,Volume
0,2020-01-02,FTNT,21.936001,21.936001,22.058001,21.476000,21.520000,5541000.0
1,2020-01-03,FTNT,22.242001,22.242001,22.306000,21.620001,21.684000,6641000.0
2,2020-01-06,FTNT,22.400000,22.400000,22.472000,21.906000,22.122000,7369000.0
3,2020-01-07,FTNT,22.378000,22.378000,22.502001,22.214001,22.466000,3796500.0
4,2020-01-08,FTNT,22.733999,22.733999,22.879999,22.360001,22.389999,6953000.0
...,...,...,...,...,...,...,...,...
432229,2022-07-25,AIH,2.220000,2.220000,2.360000,2.110000,2.310000,6400.0
432230,2022-07-26,AIH,2.370000,2.370000,2.370000,2.360000,2.360000,2500.0
432231,2022-07-27,AIH,2.400000,2.400000,2.400000,2.234000,2.350000,20400.0
432232,2022-07-28,AIH,2.340000,2.340000,2.370000,2.270000,2.270000,2800.0


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432234 entries, 0 to 432233
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date       432234 non-null  object 
 1   Index      432234 non-null  object 
 2   Adj Close  423623 non-null  float64
 3   Close      423623 non-null  float64
 4   High       423623 non-null  float64
 5   Low        423623 non-null  float64
 6   Open       423623 non-null  float64
 7   Volume     423623 non-null  float64
dtypes: float64(6), object(2)
memory usage: 26.4+ MB


In [51]:
stock_list = data['Index'].unique()
print(len(stock_list))

666


## High_Low & Std (last 21 index == 1 month)

In [57]:
def high_low_m(df, m):
    start_index = df.index[0]
    end_index = df.index[-1]
    #print(f'start_index: {start_index}; end_index: {end_index}')
    
    high_low_list = []
    duration = None
    
    if m == 1:
        duration = 21
    elif m == 2:
        duration = 42
    else:
        duration = 63
        
    for idx in df.index:
        if idx-start_index < (duration-1):
            high_low_list.append(None)
        else:
            df_last_month = df[idx-(duration-1): idx+1]
            assert len(df_last_month) == duration
            High_highest = max(df_last_month['High'])
            Low_lowest = min(df_last_month['Low'])
            high_low_list.append(High_highest/Low_lowest)

    assert len(high_low_list) == df.shape[0]
    return high_low_list


In [58]:
def std_m(df, m):
    start_index = df.index[0]
    end_index = df.index[-1]
    #print(f'start_index: {start_index}; end_index: {end_index}')
    
    std_list = []
    RoR_list = test['Adj Close'].pct_change()
    duration = None
    
    if m == 1:
        duration = 21
    elif m == 2:
        duration = 42
    else:
        duration = 63
    
    for idx in df.index:
        if idx-start_index < duration:
            std_list.append(None)
        else:
            RoR_last_month = RoR_list[idx-(duration-1): idx+1]
            assert len(RoR_last_month) == duration
            std_list.append(np.std(RoR_last_month))
    
    assert len(std_list) == df.shape[0]
    return std_list 

In [68]:
def process(data):
    stock_list = data['Index'].unique()
    high_low_1m_total = []
    high_low_2m_total = []
    high_low_3m_total = []
    std_1m_total = []
    std_2m_total = []
    std_3m_total = []
    for stock in stock_list:
        print('Processing stock: ', stock)
        df_tmp = data.loc[data['Index'] == stock].copy().reset_index(drop=True)
        high_low_1m = high_low_m(df_tmp, 1)
        high_low_2m = high_low_m(df_tmp, 2)
        high_low_3m = high_low_m(df_tmp, 3)
        std_1m = std_m(df_tmp, 1)
        std_2m = std_m(df_tmp, 2)
        std_3m = std_m(df_tmp, 3)
        
        high_low_1m_total += high_low_1m
        high_low_2m_total += high_low_2m
        high_low_3m_total += high_low_3m
        std_1m_total += std_1m
        std_2m_total += std_2m
        std_3m_total += std_3m
    
    assert len(high_low_1m_total) == data.shape[0]
    assert len(high_low_2m_total) == data.shape[0]
    assert len(high_low_3m_total) == data.shape[0]
    assert len(std_1m_total) == data.shape[0]
    assert len(std_2m_total) == data.shape[0]
    assert len(std_3m_total) == data.shape[0]
    
    data['high_low_1m'] = high_low_1m_total
    data['high_low_2m'] = high_low_2m_total
    data['high_low_3m'] = high_low_3m_total
    data['std_1m'] = std_1m_total
    data['std_2m'] = std_2m_total
    data['std_3m'] = std_3m_total


In [61]:
process(data)
data

Processing stock:  FTNT
Processing stock:  LAND
Processing stock:  GDYN
Processing stock:  REG
Processing stock:  ODFL
Processing stock:  SYNA
Processing stock:  PRFT
Processing stock:  IMKTA
Processing stock:  METC
Processing stock:  PTSI
Processing stock:  CLMT
Processing stock:  MCBS
Processing stock:  GAIN
Processing stock:  SBNY
Processing stock:  KRUS
Processing stock:  LKQ
Processing stock:  COKE
Processing stock:  EXLS
Processing stock:  PCB
Processing stock:  INMD
Processing stock:  WIRE
Processing stock:  CCRN
Processing stock:  PAYX
Processing stock:  IRMD
Processing stock:  AEHR
Processing stock:  RELL
Processing stock:  RRR
Processing stock:  AFBI
Processing stock:  INTU
Processing stock:  CCB
Processing stock:  MSFT
Processing stock:  GOOGL
Processing stock:  TBK
Processing stock:  DOGZ
Processing stock:  GOOD
Processing stock:  BWFG
Processing stock:  GOOG
Processing stock:  NVDA
Processing stock:  ATLC
Processing stock:  BBCP
Processing stock:  BCYC
Processing stock:  C

Processing stock:  AXON
Processing stock:  OVID
Processing stock:  CAN
Processing stock:  EMKR
Processing stock:  DSWL
Processing stock:  RUBY
Processing stock:  DLTR
Processing stock:  BFIN
Processing stock:  MBWM
Processing stock:  SLGG
Processing stock:  TTD
Processing stock:  IROQ
Processing stock:  GILD
Processing stock:  FSEA
Processing stock:  JNCE
Processing stock:  RDCM
Processing stock:  ATRC
Processing stock:  SANM
Processing stock:  EBC
Processing stock:  ALNY
Processing stock:  SPSC
Processing stock:  LTCHW
Processing stock:  HRZN
Processing stock:  ARGX
Processing stock:  ZUMZ
Processing stock:  GIFI
Processing stock:  WING
Processing stock:  NFLX
Processing stock:  ABCM
Processing stock:  GPRO
Processing stock:  KRNY
Processing stock:  FHTX
Processing stock:  KRTX
Processing stock:  CVLT
Processing stock:  ADBE
Processing stock:  LIZI
Processing stock:  PCTY
Processing stock:  FGF
Processing stock:  AUDC
Processing stock:  TACT
Processing stock:  ACET
Processing stock:  

,Date,Index,Adj Close,Close,High,Low,Open,Volume,high_low_1m,high_low_2m,high_low_3m,std_1m,std_2m,std_3m
0,2020-01-02,FTNT,21.936001,21.936001,22.058001,21.476000,21.520000,5541000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-03,FTNT,22.242001,22.242001,22.306000,21.620001,21.684000,6641000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-06,FTNT,22.400000,22.400000,22.472000,21.906000,22.122000,7369000.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-07,FTNT,22.378000,22.378000,22.502001,22.214001,22.466000,3796500.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-08,FTNT,22.733999,22.733999,22.879999,22.360001,22.389999,6953000.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432229,2022-07-25,AIH,2.220000,2.220000,2.360000,2.110000,2.310000,6400.0,1.443066,1.641026,2.169492,0.024337,0.027798,0.032932
432230,2022-07-26,AIH,2.370000,2.370000,2.370000,2.360000,2.360000,2500.0,1.430168,1.641026,2.169492,0.028452,0.030331,0.033389
432231,2022-07-27,AIH,2.400000,2.400000,2.400000,2.234000,2.350000,20400.0,1.430168,1.641026,2.169492,0.029225,0.030657,0.032979
432232,2022-07-28,AIH,2.340000,2.340000,2.370000,2.270000,2.270000,2800.0,1.430168,1.641026,2.169492,0.029100,0.030677,0.033036


In [66]:
data_final = data.copy()
indexes = data_final[data_final['Date'] <= '2020-12-31'].index
data_final.drop(indexes, inplace = True)
data_final.to_csv('data/data_volatility_raw.csv', index=False) 
data_final

,Date,Index,Adj Close,Close,High,Low,Open,Volume,high_low_1m,high_low_2m,high_low_3m,std_1m,std_2m,std_3m
253,2021-01-04,FTNT,29.112,29.112,29.931999,28.572001,29.914000,5391000.0,1.246585,1.403221,1.436347,0.023125,0.021400,0.023501
254,2021-01-05,FTNT,28.348,28.348,28.535999,27.966000,28.312000,6130000.0,1.234939,1.394797,1.436347,0.023887,0.021925,0.023665
255,2021-01-06,FTNT,27.622,27.622,28.122000,27.462000,27.464001,6531000.0,1.234939,1.394797,1.436347,0.024728,0.021979,0.023906
256,2021-01-07,FTNT,28.650,28.650,28.724001,27.822001,27.826000,6856000.0,1.234939,1.394797,1.436347,0.025711,0.022255,0.024196
257,2021-01-08,FTNT,29.628,29.628,29.698000,28.818001,28.902000,6301000.0,1.234939,1.394797,1.436347,0.025998,0.022382,0.024500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432229,2022-07-25,AIH,2.220,2.220,2.360000,2.110000,2.310000,6400.0,1.443066,1.641026,2.169492,0.024337,0.027798,0.032932
432230,2022-07-26,AIH,2.370,2.370,2.370000,2.360000,2.360000,2500.0,1.430168,1.641026,2.169492,0.028452,0.030331,0.033389
432231,2022-07-27,AIH,2.400,2.400,2.400000,2.234000,2.350000,20400.0,1.430168,1.641026,2.169492,0.029225,0.030657,0.032979
432232,2022-07-28,AIH,2.340,2.340,2.370000,2.270000,2.270000,2800.0,1.430168,1.641026,2.169492,0.029100,0.030677,0.033036
